In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam


In [2]:
# Load the IMDb dataset
vocab_size = 10000  # Limit to the top 10,000 words
max_len = 300  # Maximum length of review (in words)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform input size
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)


17464789/17464789 [==============================] - 2s 0us/step


In [3]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [4]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
313/313 [==============================] - 34s 95ms/step - loss: 0.4330 - accuracy: 0.7986 - val_loss: 0.3239 - val_accuracy: 0.8716
Epoch 2/5
313/313 [==============================] - 17s 54ms/step - loss: 0.2404 - accuracy: 0.9068 - val_loss: 0.3031 - val_accuracy: 0.8750
Epoch 3/5
313/313 [==============================] - 12s 40ms/step - loss: 0.1669 - accuracy: 0.9383 - val_loss: 0.3614 - val_accuracy: 0.8612
Epoch 4/5
313/313 [==============================] - 10s 33ms/step - loss: 0.1123 - accuracy: 0.9610 - val_loss: 0.3951 - val_accuracy: 0.8628
Epoch 5/5
313/313 [==============================] - 10s 32ms/step - loss: 0.1106 - accuracy: 0.9603 - val_loss: 0.4450 - val_accuracy: 0.8390


In [5]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc}')


782/782 [==============================] - 7s 9ms/step - loss: 0.4597 - accuracy: 0.8336
Test Accuracy: 0.8335999846458435


In [6]:
predictions = model.predict(x_test)
# Convert predictions to binary labels
predicted_labels = (predictions > 0.5).astype(int)

# Display some predictions
for i in range(5):
    print(f'Review: {x_test[i]}')
    print(f'Prediction: {"Positive" if predicted_labels[i][0] == 1 else "Negative"}')
    print(f'Actual: {"Positive" if y_test[i] == 1 else "Negative"}\n')


782/782 [==============================] - 7s 8ms/step
Review: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    

In [8]:
import pickle

# Save the model architecture and weights as a single HDF5 file
model.save('sentiment_analysis_model.h5')

# Save the model architecture to JSON format
model_json = model.to_json()

# Save the architecture and weights to a pickle file
with open('sentiment_analysis_model.pkl', 'wb') as file:
    pickle.dump({'model_json': model_json, 'model_weights': model.get_weights()}, file)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
